# 移動ライブラリ（rosnav）の使い方
rosnavという自律移動機能を使って、ロボットを指定した場所まで障害物を避けながら移動させてみましょう。

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>rosnavを使った絶対座標指定による移動をスクリプトを用いて実行する方法を学習します</p>
</div> 

## 1. 準備

docker-compose upにてシミュレータ環境を実行したのち、ブラウザから
http://localhost:3000/
にアクセスし、シミュレータ画面を開きましょう。gazeboというシミュレータの画面が表示されました。
![title](./imgs/gazebo.png)
次に、gazebo上のシミュレーション開始ボタンをクリックして、シミュレーションを開始しましょう。シミュレーション開始ボタンは左下の矢印です。
![title](./imgs/play_button.png)
シミュレーションが開始されました。マウス操作で視点の位置や角度を変えてみましょう。
![title](./imgs/gazebo_view_change.png)


## 2. rviz
次に、rvizを起動します。rvizはロボットのセンサの状態等を可視化するためのツールです。下記コマンドを実行して、起動してください。

In [ ]:
%%script bash --bg
rviz -d data/nav.rviz > /dev/null 2>&1

シミュレータ画面で、rvizが立ち上がったことを確認してください。rvizでは、ロボットに搭載された各種センサ情報を確認できます。

![title](./imgs/rviz.png)

gazeboとrvizの画面を切り替えるには、画面下の[Gazebo]や[nav.rviz]をクリックしましょう。


![title](./imgs/change_window.png)


## 3. rvizを使った自律移動

rvizから自律移動のゴールを入力します。rvizの上部のツールバーにある「2D Nav Goal」をクリックし、部屋内の障害物のない場所を指定してみましょう
![title](./imgs/2d_nav_goal_button.png)
![title](./imgs/2d_nav_goal.png)
HSRが自律移動を開始します。rviz画面や、シミュレータ画面で、HSRが障害物を避けながら指定した位置に移動することを確認しましょう。
- rviz画面
![title](./imgs/moving.png)
- gazebo画面
![title](./imgs/moving_gazebo.png)

## 4. 障害物回避
次に、人の位置を移動させてみましょう。gazeboの上にある
![title](./imgs/move_object_button.png)
このボタンをクリックしてから、人をクリックしてください。
![title](./imgs/human_select.png)
青、赤、緑の矢印が表示されました。赤と緑の矢印を動かすと、人をXY平面上にて動かせます。人を動かしている時に、人をロボットにぶつけてまうと、ロボットの挙動がおかしくなるので、ぶつからないように移動させましょう。

終わったら、矢印ボタンを押して、選択状態を終了しましょう。
![title](./imgs/back_to_default.png)

rvizの画面に戻って、「2D Nav Goal」を与え、HSRを自律移動させましょう。
![title](./imgs/move_avoiding_human.png)
人をよけながら自律移動できたでしょうか？

# 5. pythonスクリプトからの自律移動

前章はrvizから自律移動を実行しましたが、今回はPythonから指定した座標に自律移動する指令を出してみましょう。

### 1. 準備

自律移動機能を使うには、ROSのアクションの仕組みを使います。

必要なライブラリをインポートし、初期化します。

In [4]:
import rospy
import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal

# ROSの初期化
rospy.init_node('send_goal')

# Actionの初期化
client = actionlib.SimpleActionClient('/move_base', MoveBaseAction)
client.wait_for_server()

### 2. ゴール座標指定
次に、ゴールの座標を決めましょう。rvizの右上の「Publish Point」ボタンをクリックします。
![title](./imgs/publish_point_button.png)
次に、地図上で目的地にマウスを持って行くと、rvizの左下の「Select this point」の部分に座標が表示されます。この例では、x=1.29,y=3.13,z=0.00747 が目的地の座標です。zは不要なので、x,yの値をメモしておきましょう。
![title](./imgs/point_values.png)

#### 注意: ゴールを地図から外れた位置に設定すると誤動作するので、地図内の位置にしてください。

move_baseアクションのゴール設定に使うMoveBaseGoal型の変数を作成します。

内容を見ると、位置（position）と向き（orientation）からなる構造体のようです。

In [6]:
goal = MoveBaseGoal()
goal

target_pose: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  pose: 
    position: 
      x: 0.0
      y: 0.0
      z: 0.0
    orientation: 
      x: 0.0
      y: 0.0
      z: 0.0
      w: 0.0

まずは、frame_idに「map」を設定します。

これは、「ここで設定するゴールはmap座標系上の座標として定義する」という意味です（ROSは複数の座標系を同時に扱うことができます）。

**この設定をしないと移動が実行されないので忘れずに設定してください**

In [7]:
goal.target_pose.header.frame_id = "map"

次に目的地のxy座標を設定します。

In [9]:
goal.target_pose.pose.position.x = 1.29
goal.target_pose.pose.position.y = 3.13

最後に、目標姿勢（ロボットの向き）を設定します。

ROSでは、姿勢の表現には「四元数（quaternion）」が多く使われます。

我々がこれまで良く使ってきた姿勢の表現は「オイラー角」でした。
オイラー角はロール・ピッチ・ヨーの3次元で姿勢を表現しますが、四元数はxyzwの4次元で姿勢を表現します。
オイラー角より1次元分表現能力が高いため、オイラー角を利用した場合に発生する「ジンバルロック」などの問題を回避することができます。

ただし、四元数は人間にとって直感的に理解しにくいため、ここでは姿勢の設定はオイラー角で行い、プログラム上で四元数に変換しています。

In [10]:
import math
from tf import transformations

theta = 10  # deg
q = transformations.quaternion_from_euler(0.0, 0.0, theta / 180.0 * math.pi)

goal.target_pose.pose.orientation.x = q[0]
goal.target_pose.pose.orientation.y = q[1]
goal.target_pose.pose.orientation.z = q[2]
goal.target_pose.pose.orientation.w = q[3]

ゴール情報がすべて設定できたので、内容を確認しましょう。

In [11]:
goal

target_pose: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "map"
  pose: 
    position: 
      x: 1.29
      y: 3.13
      z: 0.0
    orientation: 
      x: 0.0
      y: 0.0
      z: 0.0871557427477
      w: 0.996194698092

## 3. ゴールを送る
ゴールをロボットに送りましょう。下記のコマンドを実行し、シミュレータ上でのロボットの動作を観察してください。

In [ ]:
# goal座標をロボットに送る
client.send_goal(goal)
finished = client.wait_for_result()

## 4. 関数化
上記の処理を関数にして簡単に呼び出せるようにしましょう。

In [6]:
import rospy
import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import math
from tf import transformations

# ROSの初期化
rospy.init_node('send_goal')

# Actionの初期化
navclient = actionlib.SimpleActionClient('/move_base', MoveBaseAction)
navclient.wait_for_server()

def move(x, y, theta):
    goal = MoveBaseGoal()
    goal.target_pose.header.frame_id = "map"
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    q = transformations.quaternion_from_euler(0.0, 0.0, theta / 180.0 * math.pi)
    goal.target_pose.pose.orientation.x = q[0]
    goal.target_pose.pose.orientation.y = q[1]
    goal.target_pose.pose.orientation.z = q[2]
    goal.target_pose.pose.orientation.w = q[3]

    # goal座標をロボットに送る
    navclient.send_goal(goal)
    return navclient.wait_for_result()

上の関数を使うと、以下のコマンドでx=1メートル, y=0.5メートル, theta=10度の位置姿勢に移動できます。

In [9]:
move(1, 0.5, 10)

True

以下のコマンドで原点に戻ります。

In [10]:
move(0, 0, 0)

True

## 5. 応用
下記のようなことを試して見ましょう。
- ある場所についたら、次のゴールに自動的に向かうようにしてみましょう
- goalの配列を作って、数カ所を巡回させましょう
- 障害物の上にゴールを置くとどうなるでしょうか？
- 自律移動中に人を動かして、ロボットの邪魔をするとどうなるでしょうか？
- 地図の外にゴールを置くとどうなるでしょうか
